In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

files.upload()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json  ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d wilmerarltstrmberg/recipe-dataset-over-2m

100% 634M/635M [00:32<00:00, 36.5MB/s]
100% 635M/635M [00:32<00:00, 20.3MB/s]


In [ ]:
!unzip -q recipe-dataset-over-2m.zip

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("recipes_data.csv")

In [ ]:
data.head()

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com


In [ ]:
len(data)

2231142

In [ ]:
#randomly drop 1M entries
from numpy.random import default_rng

arr_indices_top_drop = default_rng().choice(data.index, size=1000000, replace=False)
data = data.drop(index=arr_indices_top_drop)

In [ ]:
#get rid of useless attributes: site, source, link
data = data.drop(['site', 'source', 'link'], axis=1)

In [ ]:
data.shape

(1231142, 4)

In [ ]:
data.head()

,title,ingredients,directions,NER
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""pepper"", ""cream cheese"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken gravy"", ""cream of mushroom soup"", ""c..."
7,Scalloped Corn,"[""1 can cream-style corn"", ""1 can whole kernel...","[""Mix together both cans of corn, crackers, eg...","[""egg"", ""pepper"", ""crackers"", ""cream-style cor..."
8,Nolan'S Pepper Steak,"[""1 1/2 lb. round steak (1-inch thick), cut in...","[""Roll steak strips in flour."", ""Brown in skil...","[""oil"", ""tomatoes"", ""green peppers"", ""water"", ..."
10,Double Cherry Delight,"[""1 (17 oz.) can dark sweet pitted cherries"", ...","[""Drain cherries, measuring syrup."", ""Cut cher...","[""flavor gelatin"", ""dark sweet pitted cherries..."


In [ ]:
import ast
# Convert the string representation of the list into an actual list
data['ingredients'] = data['ingredients'].apply(ast.literal_eval)

In [ ]:
data['NER'] = data['NER'].apply(ast.literal_eval)

In [ ]:
#get all unique ingredients
ingredients_vocab = set(ingredient for sublist in data['NER'] for ingredient in sublist)

In [ ]:
len(ingredients_vocab)

135879

In [ ]:
#split the data: 80% in the training dataset, 10% in the development dataset, and 10% in the testing dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and temp (20%)
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# Split the temp data into development (50%) and testing (50%)
dev_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Display the shapes of the resulting datasets
print("Training dataset shape:", train_data.shape)
print("Development dataset shape:", dev_data.shape)
print("Testing dataset shape:", test_data.shape)

Training dataset shape: (984913, 4)
Development dataset shape: (123114, 4)
Testing dataset shape: (123115, 4)


In [ ]:
print("All dataset shape:", data.shape)


All dataset shape: (1231142, 4)


In [ ]:
#DO NOT RE-RUN!!!
# test_data.to_csv('/content/drive/My Drive/test_data.csv', index=False)

In [ ]:
#DO NOT RE-RUN!!!
# data.to_csv('/content/drive/My Drive/spring24/NLP499 group/data/data.csv', index=False)

In [ ]:
#DO NOT RE-RUN!!!
# dev_data.to_csv('/content/drive/My Drive/spring24/NLP499 group/data/dev_data.csv', index=False)

In [ ]:
#DO NOT RE-RUN!!!
# train_data.to_csv('/content/drive/My Drive/spring24/NLP499 group/data/train_data.csv', index=False)

In [ ]:
input_query = "what dish I can make with tomato, potato, egg"

# Extract ingredients from the input query
input_ingredients = set(input_query.replace(',', '').split()[6:])


In [ ]:
def find_dishes(input_ingredients, data):
    dishes_directions = []
    for index, row in dev_data.iterrows():
        if all(any(ingredient_part in dish_ingredient for dish_ingredient in row['NER'] for ingredient_part in input_ingredient.split()) for input_ingredient in input_ingredients):
            dishes_directions.append((row['title'], row['ingredients'], row['directions']))
    return dishes_directions

In [ ]:
# suggested_dishes = find_dishes(input_ingredients, data)
suggested_dishes = find_dishes(input_ingredients, data)[:50] # get the first 50 suggestions
print(f"You can make these dishes with the ingredients you have: \n {suggested_dishes}" )
for dish in suggested_dishes:
  print("Dish: ", dish[0])
  print("--Ingredients: ", dish[1])
  print("--Recipe: ", dish[2])

You can make these dishes with the ingredients you have: 
 [('Ham and Spinach Hash with Fried Eggs', ['2 tablespoons extra-virgin olive oil (EVOO) (twice around the pan)', '4 tablespoons unsalted butter', '8 small red potatoes', 'Salt and freshly ground black pepper', '1 tablespoon fresh thyme leaves, chopped (from 4 sprigs)', '1/4 teaspoon crushed red pepper flakes', '1 medium red onion, chopped', '1 large ham steak, finely chopped', '4 extra-large eggs', '2 cups baby spinach leaves, chopped', '10 fresh basil leaves, chopped or torn', '1/2 cup grated Parmigiano-Reggiano (a couple of generous handfuls)', '2 plum tomatoes, seeded and chopped'], '["Preheat a large nonstick skillet over medium-high heat with the EVOO and 2 tablespoons of the butter.", "While the pan is getting hot and the butter is melting, cut the potatoes in half and then thinly slice them.", "Add them to the hot skillet and season with salt, pepper, the thyme, and the red pepper flakes.", "Cook, stirring every now and 

In [ ]:
data['NER'] = data['NER'].replace(',',' , ').replace('.',' . ')